<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#0e4be5;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Compute Hessian Matrix for Optimized TS </h1>
<br><br>
</center>

In [5]:
import qcportal as ptl

## Define Variables:

In [13]:
ts_entry_name = 'ts1_02'
username = None
password = None
ts_from_neb_col_name = "salen_d-lac_ts_from_neb"
opt_model_chem = 'b3lyp-d3_def2-svp'

In [9]:
client = ptl.FractalClient(address="152.74.10.245:7778", username = username, password = password, verify=False)

## A) Get Optimized Molecule ID 

In [10]:
ds_opt = client.get_collection("OptimizationDataset", "salen_d-lac_ts_from_neb")
ds_opt.status(collapse = False)

,b3lyp-d3bj_def2-svp,b3lyp-d3_def2-svp
ts1_02,None,COMPLETE
ts1_02_G2,None,COMPLETE
ts1_02_G1,None,COMPLETE
ts1_03,None,COMPLETE
ts1_04,None,COMPLETE
ts2_02,None,COMPLETE
ts1_01,None,COMPLETE


In [14]:
rr = ds_opt.get_record("ts1_01", specification = opt_model_chem)
mol_opt_ts = rr.get_final_molecule()
mol_opt_ts

NGLWidget()

In [12]:
ts_id = mol_opt_ts.id
ts_id

'30064'

## A) Send Hessian Computation 

In [15]:
#Send the computation to the client

manager_tag = 'tera'

kw = ptl.models.KeywordSet(id = 1, **{"values": {"dftd": "d3", "dftgrid" : 2, "convthre" : "3.0e-7", "threall": "1.0e-13", "scf" : "diis+a"}})
kw_id = client.add_keywords([kw])[0]

method, basis = opt_model_chem.split('_')

rc = client.add_compute(program = "terachem", 
                       method = method.split('-')[0], 
                       basis = basis, 
                       driver = "hessian", 
                       molecule = ts_id, 
                       tag= manager_tag,
                       keywords = kw_id,
                       full_return = True)
rc

TaskQueuePOSTResponse(meta=ResponsePOSTMeta(errors=[], success=True, error_description='False', n_inserted=1, duplicates=[], validation_errors=[]), data=<ComputeResponse(nsubmitted=0 nexisting=1)>)

## C) Check Hessian Computation 

You need to run the above line first to check the state of the computation. If you already ran it once it won't run it again, just get the job id. 

In [17]:
#  Check Hessian job status
cr = rc.data
ids = cr.dict()['ids']
rr_h = client.query_results(ids)[0]
rr_h.dict()['status']

<RecordStatusEnum.complete: 'COMPLETE'>